<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [23]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install pandas
# !pip install nltk

In [1]:
import os
import re
import csv
import json
import hazm
import nltk
import pickle
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [2]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [3]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            category_PER = soup.find_all('li', {'class': 'breadcrumb-item'})
            category_PER = list(map(lambda x: x.text.strip(), category_PER))[1:]
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category-PER': category_PER,
                'category-ENG': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [4]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [5]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = read_dataset_from_file()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [6]:
def display_dataset_info():
    global CATEGORIES

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for data in dataset:
        length_dict[data['category-ENG']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,80986
1,سیاسی,15810
2,جهانی,4800
3,اقتصادی,9873
4,اجتماعی,14070
5,شهری,4260
6,ورزشی,8421
7,علمی,3267
8,فرهنگی,6657
9,فناوری اطلاعات,444


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [7]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم.
    </p>
</div>

In [8]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()

def get_preprocessed_texts(texts, mode):
    if mode == 'save':
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
        with open('preprocessed_texts.json', 'w', encoding='utf-8') as file:
            json.dump(preprocessed_texts, file)
            return preprocessed_texts
    if mode == 'load':
        with open('preprocessed_texts.json', encoding='utf-8') as file:
            preprocessed_texts = json.load(file)
            return preprocessed_texts

texts = [data_to_text(data) for data in dataset]
preprocessor = Preprocessor(stopwords_path='stopwords.txt')
preprocessed_texts = get_preprocessed_texts(texts, mode='load')

In [11]:
def display_most_frequent_words(words):
    most_frequent_words = nltk.FreqDist(words).most_common(50)
    word_list, frequency_list = tuple(map(list, zip(*most_frequent_words)))
    freq_dict = dict(لغت=word_list, تکرار=frequency_list)
    display(pd.DataFrame(freq_dict))

words = [word for text in tqdm(preprocessed_texts) for word in text]
display_most_frequent_words(words)

100%|██████████| 80986/80986 [00:01<00:00, 76770.36it/s]


,لغت,تکرار
0,ایران,115931
1,سال,112856
2,کشور,105230
3,گزارش,95160
4,هزار,87478
5,قرار,77185
6,تهران,73481
7,قیمت,68097
8,روز,67339
9,افزایش,66540


## Boolean

In [60]:
class BooleanIR:

    def __init__(self):
        self.vectorizer = CountVectorizer(binary=True)
        self.vectors = None
        self.feature_names = None
        self.df = None
    
    def fit_transform_vectoriazer(self, dataset):
        self.vectors = self.vectorizer.fit_transform([' '.join(row) for row in dataset])
        self.feature_names = self.vectorizer.get_feature_names_out()
        dense = self.vectors.todense()
        denselist = dense.tolist()
        self.df = pd.DataFrame(denselist, columns=self.feature_names)
    
    def predict(self, query, dataset, k):
        query_transform = self.vectorizer.transform([query])
        query_transform = query_transform.todense()
        query_transform = query_transform.tolist()
        query_transform = query_transform[0]
        query_index = np.nonzero(query_transform)[0]
        query_result = []
        for index, row in self.df.iterrows():
            if all([row[i] for i in query_index]):
                query_result.append(index)
            if len(query_result) == k:
                break
        return dataset.iloc[query_result]
        

    def cosine_sim(self, query, row):
        return np.dot(query, row) / (np.linalg.norm(query) * np.linalg.norm(row))

    def save_model(self, path="boolean_model.pickle"):
        pickle.dump(self, open(path, "wb"))


In [61]:
boolean_model = BooleanIR()
boolean_model.fit_transform_vectoriazer(preprocessed_texts[67000:69000])
boolean_model.save_model()

In [66]:
query = "انقلاب رهبر"
boolean_model.predict(query, dataset, 7)

,date,title,intro,body,category-PER,category-ENG
67033,پنجشنبه ۵ خرداد ۱۴۰۱ - ۱۷:۱۶,تصاویر و ویدئو | اجتماع ۱۰۰ هزار نفری «سلام فر...,در حالی که بیش از ۲۴۰ هزار نفر برای حضور در اج...,به گزارش همشهری آنلاین، اجتماع خانوادگی «سلام ...,"['فرهنگ', 'فرهنگ عمومی']",Culture
67084,چهارشنبه ۴ خرداد ۱۴۰۱ - ۰۷:۱۰,تصویری متفاوت از رهبر انقلاب در کنار ‌محمد جها...,با قاب تاریخ به ایران قدیم سفر و یادی ‌از ‌گذش...,به گزارش همشهری آنلاین به نقل از هفت صبح، عکس...,"['فرهنگ', 'تاريخ']",Culture
67111,دوشنبه ۲ خرداد ۱۴۰۱ - ۱۸:۱۰,چین ۷۰ هزار سالن سینما دارد ایران ۶۰۰ تا!,وزیر ارشاد با بیان اینکه گسترش صادرات محصولات ...,به گزارش همشهری آنلاین، نخستین نشست «شورای توس...,"['فرهنگ', 'سینما']",Culture
67675,یکشنبه ۲۹ فروردین ۱۴۰۰ - ۱۲:۵۳,بازنشر | همسر نواب صفوی : سرباز سید مجتبی بودم,همسر نواب صفوی: من وقتی ازدواج کردم به‌زعم خود...,همشهری- ابراهیم جلالیان: شاید گفت‌وگو با همسر ...,"['زندگی', 'چهره\u200cها']",LifeSkills
68388,شنبه ۲۵ اردیبهشت ۱۴۰۰ - ۱۹:۳۲,کنایه فتاح به کدام نامزد بود؟,پرویز فتاح، رئیس بنیاد مستضعفین، در توئیتی نسب...,به گزارش همشهری‌آنلاین، پرویز فتاح در توئیتی ن...,"['سياست', 'سیاست داخلی']",LifeSkills
68630,جمعه ۱۷ اردیبهشت ۱۴۰۰ - ۱۶:۴۱,لزوم همکاری با خارجی‌ها در خودروسازی بر اساس ا...,دبیر انجمن صنایع همگن نیرو محرکه و قطعه‌سازان ...,به گزارش همشهری آنلاین به نقل از ایرنا، «آرش م...,"['اقتصاد', 'خودرو']",LifeSkills
68681,چهارشنبه ۱۵ اردیبهشت ۱۴۰۰ - ۱۲:۳۱,وزیر کار: حقوق بازنشستگان تأمین اجتماعی ۱۳۲ در...,وزیر تعاون، کار و رفاه اجتماعی گفت: از اسفند ۹...,به گزارش همشهری‌آنلاین از وزارت تعاون، کار و ر...,"['اقتصاد', 'اقتصاد كلان']",LifeSkills


## TF-IDF

In [9]:
dataset = pd.DataFrame(dataset)
dataset

,date,title,intro,body,category-PER,category-ENG
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,"['سياست', 'سیاست\u200cخارجی']",Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,"['سياست', 'سیاست\u200cخارجی']",Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,"['جهان', 'آمریکا']",Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,"['سياست', 'سیاست داخلی']",Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,"['سياست', 'سیاست داخلی']",Politics
...,...,...,...,...,...,...
80981,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۵۴,از زینک چه می‌دانید؟ | علائم کمبود زینک یا روی,بدن شما نیاز به انواع مواد مغذی دارد تا عملکرد...,به گزارش همشهری آنلاین، بدن شما نیاز به انواع ...,"['زندگی', 'تندرستی']",LifeSkills
80982,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۰۲,یک راهکار ساده برای مطمئن شدن از کیفیت عینک آف...,استفاده از عینک آفتابی یک کار ضروری برای محافظ...,به گزارش همشهری آنلاین، بررسی‌ها نشان داده قرا...,"['زندگی', 'مهارت\u200cهای زندگی']",LifeSkills
80983,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۸:۰۰,تصاویری از سرزمین لک‌لک‌های ایران | زریوار میز...,بهار فصل خوبی برای سفر به کردستان است؛ جاده‌ها...,به گزارش همشهری آنلاین، شنیدن صدای لک‌لک‌ها یک...,"['زندگی', 'پیشنهاد سفر']",LifeSkills
80984,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۷:۵۵,۱۰ ترفند خانگی که سردردتان را بدون نیاز به قرص...,سردرد مشکل نادری نیست و همه‌ ما گاهی دچارش می‌...,به گزارش همشهری آنلاین به نقل از سلامت نیوز، ت...,"['زندگی', 'تندرستی']",LifeSkills


In [35]:
class TFIDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.vectors = None
        self.feature_names = None
        self.df = None
    
    def fit_transform_vectoriazer(self, dataset):
        self.vectors = self.vectorizer.fit_transform([' '.join(row) for row in dataset])
        self.feature_names = self.vectorizer.get_feature_names_out()
        dense = self.vectors.todense()
        denselist = dense.tolist()
        self.df = pd.DataFrame(denselist, columns=self.feature_names)
    
    def predict(self, query, dataset, k):
        query_transform = self.vectorizer.transform([query])
        query_transform = query_transform.todense()
        query_transform = query_transform.tolist()
        query_transform = query_transform[0]
        df_val = self.df.values.tolist()
        df_cosine_sim = [self.cosine_sim(query_transform, row) for row in df_val]
        self.df['query_sim'] = df_cosine_sim
        res = self.df.nlargest(k, 'query_sim').index
        print(res)
        self.df = self.df.drop(columns=['query_sim'])
        return dataset.iloc[res]
        

    def cosine_sim(self, query, row):
        return np.dot(query, row) / (np.linalg.norm(query) * np.linalg.norm(row))

    def save_model(self, path="tfidf_model.pickle"):
        pickle.dump(self, open(path, "wb"))


In [47]:
dataset.iloc[66000:70000]

,date,title,intro,body,category-PER,category-ENG
66000,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۶:۳۰,خبر خوش برای علاقه مندان «باب اسفنجی»,ماجراجویی‌های «باب اسفنجی» و دوستانش یک فصل دی...,به گزارش همشهری آنلاین، ایرنا به نقل از پایگاه...,"['فرهنگ', 'رادیو و تلویزیون']",Culture
66001,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۶:۲۶,خاموشی پیش از موعد اکسپو دوبی,چراغ‌های هر اکسپوی جهانی در انتهای تاریخ برپای...,به گزارش همشهری آنلاین به نقل از ایسنا، شامگاه...,"['فرهنگ', 'دین و اندیشه']",Culture
66002,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۵:۵۳,آخرین آرزوی بهروز وثوقی چیست؟,بهروز وثوقی می‌گوید آخرین آرزوی او این است که ...,به گزارش همشهری آنلاین به نقل از چلچراغ، بازیگ...,"['فرهنگ', 'سینما']",Culture
66003,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۵:۵۲,جنجالی‌ترین انتخاب‌های اسکار | مروری‌بر ۲۰ ان...,مروری‌بر جنجالی‌ترین انتخاب‌های اسکار در حالی ...,به گزارش همشهری آنلاین، مهر به نقل از ورایتی ن...,"['فرهنگ', 'سینما']",Culture
66004,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۴:۲۷,وقتی مهران مدیری برای همسرِ یک بازیگر آرزوی صب...,مهران مدیری در برنامه «دورهمی» هنگامی که شهرام...,به گزارش همشهری‌آنلاین، ویژه‌برنامه «دورهمی» ب...,"['فرهنگ', 'رادیو و تلویزیون']",Culture
...,...,...,...,...,...,...
69995,چهارشنبه ۱۲ خرداد ۱۴۰۰ - ۱۲:۰۷,چگونه خسارت بسته های پستی‌ آسیب دیده را بگیریم؟,این روزها با توجه به افزایش خریدها و سفارش های...,به گزارش همشهری آنلاین به نقل از ایسنا، مرسوله...,"['اقتصاد', 'اقتصاد كلان']",LifeSkills
69996,چهارشنبه ۱۲ خرداد ۱۴۰۰ - ۱۲:۰۰,بعد از تزریق واکسن همچنان ناقل هستید؟ | پاسخ‌ه...,افرادی که اکنون در حال دریافت واکسن هستند این ...,همشهری- زهرا خلجی: در مراکز تزریق واکسن کشور ا...,"['جامعه', 'بهداشت و درمان']",LifeSkills
69997,چهارشنبه ۱۲ خرداد ۱۴۰۰ - ۱۱:۳۱,مصرف برق امروز رکورد می‌زند,سخنگوی صنعت برق کشور گفت: پیش بینی می‌شود که ا...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,"['اقتصاد', 'انرژی']",LifeSkills
69998,چهارشنبه ۱۲ خرداد ۱۴۰۰ - ۱۱:۲۰,تصاویر | عبدالناصر همتی با لباس و مدرک کاراته ...,تصاویری از عبدالناصر همتی در لباس کاراته منتشر...,به گزارش همشهری آنلاین همسر عبدالناصر همتی که ...,"['ورزش', 'رزمی']",LifeSkills


In [44]:
tf_idf = TFIDF()
tf_idf.fit_transform_vectoriazer(preprocessed_texts[66000:70000])
tf_idf.save_model()

  0%|          | 0/80986 [1:18:17<?, ?it/s]


In [45]:
query = "سامسونگ گلکسی"
tf_idf.predict(query, dataset, 3)

Int64Index([1389, 1279, 1218], dtype='int64')


,date,title,intro,body,category-PER,category-ENG
1389,چهارشنبه ۲۲ اردیبهشت ۱۴۰۰ - ۰۷:۰۹,تصاویری از تل‌آویو در شبی که گذشت,تل آویو در پاسخ به وحشیگری‎‌های صهیونست‌ها با ...,به گزارش همشهری آنلاین به نقل از المنار، ده‌ها...,"['جهان', 'غرب آسیا']",Politics
1279,جمعه ۲۴ اردیبهشت ۱۴۰۰ - ۰۱:۰۱,ادعای وزیر خارجه آمریکا: ایران نباید به سلاح ا...,وزیر خارجه آمریکا بدون اظهار نظر درباره اینکه ...,به گزارش همشهری‌آنلاین به نقل از ایرنا، آنتونی...,"['سياست', 'سیاست\u200cخارجی']",Politics
1218,شنبه ۲۵ اردیبهشت ۱۴۰۰ - ۱۴:۱۴,بغض جهانگیری ترکید | برای دفاع از صندوق و حق ر...,معاون اول دولت حسن روحانی که امروز اعلام نامزد...,به گزارش همشهری‌آنلاین، اسحاق جهانگیری که پس ا...,"['سياست', 'سیاست داخلی']",Politics


In [49]:
dataset.iloc[[67389, 67279, 67218]]

,date,title,intro,body,category-PER,category-ENG
67389,چهارشنبه ۱۱ اسفند ۱۴۰۰ - ۱۴:۴۷,عرضه گالکسی A13 4G و M23 5G به‌ زودی آغاز می‌ش...,سامسونگ سال گذشته گالکسی A13 5G را معرفی کرد، ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,['فناوری اطلاعات'],IT
67279,دوشنبه ۱۸ بهمن ۱۴۰۰ - ۱۲:۲۸,کارگران سامسونگ در آستانه اعتصاب | جنجال بر سر...,سامسونگ الکترونیکس ممکن است با اولین اعتصاب کا...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,['فناوری اطلاعات'],IT
67218,چهارشنبه ۱۵ دی ۱۴۰۰ - ۱۳:۰۶,یک موفقیت بزرگ برای گالکسی اس ۲۰ اف‌ای,گلکسی اس ۲۰ اف‌ای (نسخه هوادار) ثابت کرد که یک...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,['فناوری اطلاعات'],IT


## Fasttext

In [19]:
class FastText:
    
    def __init__(self, method="skipgram", k=5):
        self.method = method
        self.mean_embed = []
        self.model = None
    
    def train(self, dataset):
        with open('fasttext_train.txt', 'w') as fp:
            fp.write('\n'.join([' '.join(row) for row in dataset]))
        self.model = fasttext.train_unsupervised('fasttext_train.txt', self.method, minn=2, maxn=5)
        os.remove('fasttext_train.txt')
        for row in dataset:
            embed = np.mean([self.model.get_word_vector(x) for x in row], axis=0)
            self.mean_embed.append(embed)
        self.mean_embed  = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        query_embed = np.mean([self.model.get_word_vector(x) for x in query], axis=0)
        dataset_sim = []
        for row in self.mean_embed:
            row_cosine_sim = self.cosine_sim(query_embed, row)
            dataset_sim.append(row_cosine_sim)
        dataset_sim = np.array(dataset_sim)
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, row):
        return np.dot(query, row) / (np.linalg.norm(query) * np.linalg.norm(row))

    def save_ft_model(self, path='fasttext_model.bin'):
        self.model.save_model(path)
        np.save('fasttext_mean_embed.npy', self.mean_embed)
    
    def load_ft_model(self, path="fasttext_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('fasttext_mean_embed.npy')

In [20]:
ft = FastText()
#ft.train(preprocessed_texts)

In [21]:
#ft.save_ft_model()
ft.load_ft_model()

In [32]:
query = ' فدراسیون فوتبال اسپانیا به فرهاد مجیدی آفرین گفت '

res = ft.predict(preprocessor.preprocess(query), dataset, 5)
res

,date,title,intro,body,category-PER,category-ENG
57068,دوشنبه ۹ خرداد ۱۴۰۱ - ۰۶:۴۹,پیام تبریک سرمربی میلان ایتالیا به مجیدی و است...,سرمربی میلان به فرهاد مجیدی و بازیکنان استقلال...,به گزارش همشهری آنلاین استفانو پیولی سرمربی تی...,"['ورزش', 'فوتبال ايران']",Sport
49338,سه‌شنبه ۳ فروردین ۱۴۰۰ - ۰۹:۲۰,عکس | سلفی مجیدی با عضو جدید کادر فنی استقلال,همکار سرمربی استقلال در تیم ملی امید به کادرفن...,به گزارش همشهری‌آنلاین، فرشاد ماجدی به کادرفنی...,"['ورزش', 'فوتبال ايران']",Sport
72280,جمعه ۲۹ مرداد ۱۴۰۰ - ۲۲:۳۲,دستیار ایتالیایی مجیدی از ابتدای لیگ روی نیمکت...,گابریل پین با درخواست دوباره مجیدی به ایران خو...,به گزارش همشهری آنلاین یکی از خواسته های فرهاد...,"['ورزش', 'فوتبال ايران']",LifeSkills
51162,جمعه ۲۹ مرداد ۱۴۰۰ - ۲۲:۳۲,دستیار ایتالیایی مجیدی از ابتدای لیگ روی نیمکت...,گابریل پین با درخواست دوباره مجیدی به ایران خو...,به گزارش همشهری آنلاین یکی از خواسته های فرهاد...,"['ورزش', 'فوتبال ايران']",Sport
56582,یکشنبه ۲۵ اردیبهشت ۱۴۰۱ - ۱۳:۵۱,ناراحتی برانکو از قهرمان نشدن پرسپولیس در لیگ ...,سرمربی سابق کروات پرسپولیس به قهرمان نشدن پرسپ...,به گزارش همشهری‌آنلاین، سرمربی کروات و سابق پر...,"['ورزش', 'فوتبال ايران']",Sport


In [33]:
## TODO remove redundant news